# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [1]:
import numpy as np
import cvxopt

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        shape = np.shape(data_set)
        d = shape[1]
        kernel = np.matrix(kernel)**d
        kernel = np.array(kernel)
    return kernel

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()
data_set = iris.data

build_kernel(data_set, kernel_type='poly')

array([[2.72973418e+13, 2.57264295e+13, 2.51470225e+13, ...,
        4.16696015e+13, 4.20374312e+13, 3.91971566e+13],
       [2.57264295e+13, 2.42459217e+13, 2.36998574e+13, ...,
        3.92716193e+13, 3.96182819e+13, 3.69414580e+13],
       [2.51470225e+13, 2.36998574e+13, 2.31660925e+13, ...,
        3.83871263e+13, 3.87259806e+13, 3.61094455e+13],
       ...,
       [4.16696015e+13, 3.92716193e+13, 3.83871263e+13, ...,
        6.36095738e+13, 6.41710882e+13, 5.98353431e+13],
       [4.20374312e+13, 3.96182819e+13, 3.87259806e+13, ...,
        6.41710882e+13, 6.47375597e+13, 6.03635408e+13],
       [3.91971566e+13, 3.69414580e+13, 3.61094455e+13, ...,
        5.98353431e+13, 6.03635408e+13, 5.62850541e+13]])

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [3]:
from sklearn.model_selection import train_test_split

def choose_set_for_label(data_set, label):
    
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

    return train_data_set, test_data_set, train_labels, test_labels

In [11]:
def get_labels_count(data_set):
    labels = iris.target
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels)
    labels_count = len(train_labels)
    return labels_count

Use the code that we have implemented earlier:

In [5]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [12]:
iris = load_iris()
data_set = iris.data
labels = iris.target

data_set = data_set[labels!=2]
labels = labels[labels!=2]

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels)
objects_count = get_labels_count(data_set)
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0: -3.9881e+04 -4.0157e+04  1e+06  1e+01  2e-14
 1: -1.3449e+03 -6.9621e+04  3e+05  2e+00  2e-13
 2: -2.6954e+02 -3.7077e+04  5e+04  1e-01  1e-13
 3: -2.2516e+02 -4.0048e+03  4e+03  1e-02  3e-14
 4: -2.5456e+02 -7.0346e+02  5e+02  1e-03  3e-14
 5: -3.6139e+02 -5.5036e+02  2e+02  3e-04  7e-14
 6: -4.0821e+02 -4.1218e+02  4e+00  6e-06  4e-14
 7: -4.0998e+02 -4.1002e+02  4e-02  6e-08  6e-14
 8: -4.1000e+02 -4.1000e+02  4e-04  6e-10  3e-14
 9: -4.1000e+02 -4.1000e+02  4e-06  6e-12  3e-14
Optimal solution found.
0.45
